# Multi-Agent Systems with Strands Agents including New Loan Application Agent

This notebook demonstrates how to build a multi-agent mortgage assistant system using Strands Agents SDK. The system includes:
1. General mortgage information agent
2. Existing mortgage details agent
3. New loan application agent
4. Credit check functionality via MCP server

## 1. Setup and Installation

First, let's install the required libraries:

In [ ]:
!pip install --upgrade -q -r ../src/requirements.txt

## 2. Import Libraries and Configure Logging

In [ ]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent
import sys
from datetime import datetime, timedelta
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

from strands import Agent, tool
from strands_tools import retrieve, calculator

In [ ]:
# Set up logging specifically for Strands components
loggers = [
  'strands',
  'strands.agent',
  'strands.tools',
  'strands.models',
  'strands.bedrock'
]
for logger_name in loggers:
  logger = logging.getLogger(logger_name)
  logger.setLevel(logging.INFO)
  # Add console handler if not already present
  if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

    modelID="anthropic.claude-3-5-haiku-20241022-v1:0"

## 3. Configure Knowledge Base

First you need to create the Knowledge base before creating an agent to answer questions on mortages. To create the Amazon Bedrock Knowledge Base, you need to run this notebook first if not done already [01_create_knowledgebase.ipynb](../2_bedrock-multi-agent/01_create_knowledgebase.ipynb). When you run the below cell you should be able to see the id of the Knowledge base that you created.

In [ ]:
%store -r kb_id
print("KnowledgeBase ID:",kb_id)

In [ ]:
# Set knowledge base ID as environment variable so that Strands retrieve tool can use it
os.environ["KNOWLEDGE_BASE_ID"] = kb_id

## 4. Create Agent for General Mortgage Questions

We will be creating an agent to answer general mortage questions providing it the **retrieve** tool to access the Knowledge Base created earlier.

In [ ]:
@tool
def answer_general_mortgage_questions(query):
    # Create the General Mortgage Agent
    general_mortgage_agent = Agent(
        model=modelID,
        tools=[
           retrieve
        ],
        system_prompt="""
        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first.
        
        IMPORTANT: Always use the retrieve tool to search the knowledge base before answering any mortgage-related questions.
        
        You can:
        1. Provide general information about mortgages
        2. Handle conversations about general mortgage questions, like high level concepts of refinancing or tradeoffs of 15-year vs 30-year terms.
        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.
        4. Access a knowledge base of mortgage information using the retrieve tool
        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"
        
        When helping users:
        - ALWAYS call the retrieve tool first to search for relevant information
        - Provide clear explanations based on retrieved information
        - Use plain language to explain complex financial terms
        - Offer balanced advice considering both pros and cons
        - Be informative without making specific financial recommendations
        
        Remember that you're providing general mortgage information, not financial advice.
        Always clarify that users should consult with a financial advisor for personalized advice.
        """
    )
    return str(general_mortgage_agent(query))

Test the General agent and confirm that it consults the KB to answer the questions

In [ ]:
print(answer_general_mortgage_questions("What is the benefit of refinancing, if any?"))

## 5. Create Agent for Existing Mortgage Questions

Create the Agent for managing existing mortgages, for example you can ask when is your next payment due, etc.

In [ ]:
@tool
def get_mortgage_details(customer_id):
    # TODO: Implement real business logic to retrieve mortgage status
    return {
        "account_number": customer_id,
        "outstanding_principal": 150000.0,
        "interest_rate": 4.5,
        "maturity_date": "2030-06-30",
        "payments_remaining": 72,
        "last_payment_date": "2024-06-01",
        "next_payment_due": "2024-07-01",
        "next_payment_amount": 1250.0
    }

In [ ]:
@tool
def answer_existing_mortgage_questions(query):
    # Create the Existing Mortgage Agent
    existing_mortgage_agent = Agent(
        model=modelID,
        tools=[
           get_mortgage_details
        ],
        system_prompt="""
        You are an Existing Mortgage Assistant that helps customers with their current mortgages.

        You can:
        1. Provide information about a customer's existing mortgage
        2. Check mortgage status including balance and payment information
        3. Evaluate refinancing eligibility
        4. Calculate payoff timelines with extra payments
        5. Answer questions about mortgage terms and conditions

        When helping users:
        - Always verify the customer ID before providing information
        - Provide clear explanations of mortgage details
        - Format financial data in a readable way
        - Explain payment schedules and upcoming due dates
        - Offer guidance on refinancing options when appropriate
        - Use the knowledge base for detailed information when needed

        Remember that you're dealing with sensitive financial information, so maintain a professional tone
        and ensure accuracy in all responses.
        """
    )
    return str(existing_mortgage_agent(query))

Test the agent to check if it can answer questions about the existing mortgage.

In [ ]:
print(answer_existing_mortgage_questions("I'm customer 98991. when's my next payment due?"))

## 6. Create Agent for New Loan Applications

Now we'll create an agent to handle new mortgage loan applications.

In [ ]:
@tool
def get_mortgage_app_doc_status(customer_id: str = None):
    """Retrieves the list of required documents for a mortgage application"""
    return [
        {"type": "proof_of_income", "status": "COMPLETED"},
        {"type": "employment_information", "status": "MISSING"},
        {"type": "proof_of_assets", "status": "COMPLETED"},
        {"type": "credit_information", "status": "COMPLETED"}
    ]

@tool
def get_application_details(customer_id: str = None):
    """Retrieves details about a mortgage application"""
    return {
        "customer_id": customer_id or "123456",
        "application_id": "998776",
        "application_date": str(datetime.today() - timedelta(days=35)),
        "application_status": "IN_PROGRESS",
        "application_type": "NEW_MORTGAGE",
        "name": "Mithil"
    }

@tool
def create_customer_id():
    """Creates a new customer ID"""
    return "123456"

@tool
def create_loan_application(customer_id: str, name: str, age: int, annual_income: int, annual_expense: int):
    """Creates a new loan application"""
    print(f"Creating loan application for customer: {customer_id}")
    print(f"Customer name: {name}")
    print(f"Customer age: {age}")
    print(f"Customer annual income: {annual_income}")
    print(f"Customer annual expense: {annual_expense}")
    return f"Loan application created successfully for {name} (ID: {customer_id})"

In [ ]:
@tool
def answer_new_loan_application_questions(query):
    """Tool that handles new loan application questions using a specialized agent"""
    # Create the New Loan Application Agent
    new_loan_application_agent = Agent(
        model=modelID,
        tools=[
            get_mortgage_app_doc_status, 
            get_application_details, 
            create_customer_id, 
            create_loan_application
        ],
        system_prompt="""You are a mortgage application agent that helps customers with new mortgage applications.
        
        Instructions:
        - Greet customers warmly before answering
        - First ask for their customer ID, if they don't have one, create a new one
        - Ask for name, age, annual income, and annual expense one question at a time
        - Once you have all information, create the loan application
        - Only discuss mortgage applications, not general mortgage topics
        - Never make up information you cannot retrieve from tools"""
    )
    
    return str(new_loan_application_agent(query))

Run a simple test just to make sure the new loan application agent is running

In [ ]:
print(answer_new_loan_application_questions("Hello, I'd like to apply for a new mortgage"))

## 6.1. Interactive Chat Loop - New loan applicatoin workflow

Lets run an interactive chat loop to test the new loan application workflow - create a new customer id if new customer, ask for Name, salary, expense, etc to create the application.

In [ ]:
# Initialize conversation history
conversation_history = []

def answer_customer_query_with_history(query, history):
# Use last 5 exchanges for context
    recent_history = history[-5:] if history else []
    
    # Format conversation context
    if recent_history:
        context = '\n'.join([f"User: {h['user']}\nAssistant: {h['assistant']}" for h in recent_history])
        prompt = f"Previous conversation:\n{context}\n\nCurrent query: {query}\n\nProvide a comprehensive answer considering the conversation history."
    else:
        prompt = query
    
    return answer_new_loan_application_questions(prompt)


print("New loan application assistant - Type 'quit' to exit")
print("=" * 60)

while True:
    user_input = input("\nYou: ").strip()
    
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break
    
    if not user_input:
        continue
    
    try:
        old_stdout = sys.stdout
        sys.stdout = io.StringIO()
        
        response = answer_customer_query_with_history(user_input, conversation_history)
        
        # Store in conversation history
        conversation_history.append({
            'user': user_input,
            'assistant': str(response)
        })  
   
        # Restore stdout
        captured_output = sys.stdout.getvalue()
        sys.stdout = old_stdout
        print(f"\nAssistant: {response}")
    except Exception as e:
        print(f"\nError: {str(e)}")
        print("Please try again or type 'quit' to exit.")

## 7. Integrate Credit Check Tool via MCP Server

Now let us integrate the credit check tool provided via the MCP server. Strands includes built-in support for connecting to MCP servers and using their tools.

> 🚨 **Important Note:** Before proceeding further **Run the MCP server** provided here [creditcheck_server_http.py](../3_mcp-server-client/strands_mcp/creditcheck_server_http.py). Go to the SageMaker terminal from this notebook and navigate to the **3_mcp-server-client/strands_mcp/** folder and run the below command:

python creditcheck_server_http.py

This will start the MCP server that can accessible through this url: "http://0.0.0.0:8080/mcp".

We will connect to this server through Strands MCP client, list the tools provided by the MCP server and add the tools to the tool list provided the agent as shown int he cell below

In [ ]:
# Create MCP HTTP client
mcp_client = MCPClient(lambda: streamablehttp_client(
    url="http://0.0.0.0:8080/mcp"  # Your MCP server URL
    #streaming=True  # Enable streaming
))

## 8. Create Supervisor Agent

Create the supervisor agent and provide all specialized agent tools as well as the MCP tools. When working with MCP tools in Strands, all agent operations must be performed within the MCP client's context manager (using a with statement). This requirement ensures that the MCP session remains active and connected while the agent is using the tools.

In [ ]:
def create_supervisor_agent():
    """
    Create a supervisor agent that coordinates between the specialized agents
    and integrates MCP tools
    
    Returns:
        Agent: The supervisor agent
    """
    # Connect to MCP server and get tools
    with mcp_client:
        try:
            # Get MCP tools
            mcp_tools = mcp_client.list_tools_sync()
            logger.info(f"Loaded {len(mcp_tools)} MCP tools")
            
            # Define supervisor system prompt
            supervisor_system_prompt = """
            Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
            customer questions related to general mortgages questions, queries about existing mortgages, and new loan applications.

            For general questions, use the answer_general_mortgage_questions tool.
            For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
            For new loan applications, use the answer_new_loan_application_questions tool.
            If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
            
            You also have access to MCP tools that can perform additional function to get the credit score of existing customer.
            Use these tools when appropriate for the customer's query.
            
            Synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
            """
            
            # Combine custom tools with MCP tools
            all_tools = [
                answer_general_mortgage_questions, 
                answer_existing_mortgage_questions,
                answer_new_loan_application_questions,
                calculator
            ] + mcp_tools
            
            # Create the supervisor agent
            supervisor = Agent(
                model=modelID,
                system_prompt=supervisor_system_prompt,
                tools=all_tools
            )
            
            return supervisor
            
        except Exception as e:
            logger.error(f"Error creating supervisor agent with MCP tools: {str(e)}")
            
            # Fallback to creating supervisor without MCP tools
            logger.info("Creating supervisor agent without MCP tools")
            supervisor = Agent(
                model=modelID,
                system_prompt="""
                Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
                customer questions related to general mortgages questions, queries about existing mortgages, and new loan applications.

                For general questions, use the answer_general_mortgage_questions tool.
                For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
                For new loan applications, use the answer_new_loan_application_questions tool.
                If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
                
                Synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
                """,
                tools=[
                    answer_general_mortgage_questions, 
                    answer_existing_mortgage_questions, 
                    answer_new_loan_application_questions,
                    calculator
                ]
            )
            
            return supervisor

In [ ]:
def answer_customer_query(query):
    """
    Process the customer query through the multi-agent system with MCP integration
    
    Args:
        query: The customer's query
        
    Returns:
        str: The response from the supervisor agent
    """
    # Use MCP client context manager to ensure session remains active
    with mcp_client:
        try:
            # Get MCP tools
            mcp_tools = mcp_client.list_tools_sync()
            
            # Define supervisor system prompt
            supervisor_system_prompt = """
            Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
            customer questions related to general mortgages questions and queries about the existing mortgage.
            
            For general questions, use the answer_general_mortgage_questions tool.
            For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
            If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
            
            You also have access to MCP tools that can perform additional function to get the credit score of existing customer.
            Use these tools when appropriate for the customer's query.
            
            IMPORTANT: When using credit check tools, return ONLY the credit score value without additional analysis or explanations.
            For other queries, synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
            """
            
            # Combine custom tools with MCP tools
            all_tools = [
                answer_general_mortgage_questions, 
                answer_existing_mortgage_questions,
                answer_new_loan_application_questions,
                calculator
            ] + mcp_tools
            
            # Create the supervisor agent within the MCP context
            supervisor = Agent(
                model=modelID,
                system_prompt=supervisor_system_prompt,
                tools=all_tools
            )
            
            # Process the query within the MCP context
            return supervisor(f"Provide a comprehensive answer for this query: {query}")
            
        except Exception as e:
            print(f"Error with MCP integration: {str(e)}")
            
            # Fallback to creating supervisor without MCP tools
            supervisor = Agent(
                model=modelID,
                system_prompt="""
                Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
                customer questions related to general mortgages questions, queries about existing mortgages, and new loan applications.

                For general questions, use the answer_general_mortgage_questions tool.
                For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
                For new loan applications, use the answer_new_loan_application_questions tool.
                If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
                
                Synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
                """,
                tools=[
                    answer_general_mortgage_questions, 
                    answer_existing_mortgage_questions, 
                    answer_new_loan_application_questions,
                    calculator
                ]
            )
            
            return supervisor(f"Provide a comprehensive answer for this query: {query}")

## 9. Test the Complete Multi-Agent System

Now let's test the complete multi-agent system with various queries:

In [ ]:
# Test the supervisor agent with various queries
print("\n\nInvoking supervisor agent...\n\n")

requests = [
    "I am customer: 3345, when's my next payment due?",
    "what is my credit score, my customer id is 1111",
    "what's my balance after that payment, and what rate am I paying?",
    "why do so many people choose a 30-year mortgage??",
    "Hello, I'd like to apply for a new mortgage"
]

for request in requests:
    print(f"\n\nRequest: {request}\n\n")
    result = answer_customer_query(request)
    print(result)
    print("-" * 80)
    time.sleep(5)  # Add a small delay between requests

## 10. Interactive Chat Loop

Let's create an interactive chat loop to test the system with custom queries:

In [ ]:
# Initialize conversation history
conversation_history = []

def answer_customer_query_with_history(query, history):
    # Use last 5 exchanges for context
    recent_history = history[-5:] if history else []
    
    # Format conversation context
    if recent_history:
        context = '\n'.join([f"User: {h['user']}\nAssistant: {h['assistant']}" for h in recent_history])
        prompt = f"Previous conversation:\n{context}\n\nCurrent query: {query}\n\nProvide a comprehensive answer considering the conversation history."
    else:
        prompt = query
    
    # Use the main answer_customer_query function which handles MCP context properly
    return answer_customer_query(prompt)

print("Mortgage Assistant Chat with Memory - Type 'quit' to exit")
print("=" * 60)

while True:
    user_input = input("\nYou: ").strip()
    
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break
    
    if not user_input:
        continue
    
    try:
        response = answer_customer_query_with_history(user_input, conversation_history)
        
        # Store exchange in history
        conversation_history.append({
            'user': user_input,
            'assistant': str(response)
        })
        
        print(f"\nAssistant: {response}")
    except Exception as e:
        print(f"\nError: {str(e)}")
        print("Please try again or type 'quit' to exit.")